It is a Grid Search based SLAM implementation using LIDAR data using the following:
 - basic : we rotate the destination (later) pcd to destination's (local) grid points to score against source (former) pcd
 - adaptive grid : grid is interpolated based on velocity instead of distance ; this assumes that there are no sudden acceleration or impulsive forces or jerks
 - 2d map : creates a 2d map of the environment instead of 3d
 - parallel : the grid search for every pose has been parallelized on CPU cores

### Imports

In [1]:
import numpy as np
import pandas as pd
import gc
from tqdm import *
import os
import copy
import sys
import time
from collections import Counter, deque
from multiprocessing import Pool, cpu_count

import transforms3d

from sklearn.manifold import TSNE, LocallyLinearEmbedding
from sklearn.cluster import MeanShift

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import ipyvolume as ipv
import open3d as o3d

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
# %matplotlib notebook

### Set paths and file names

In [2]:
INVERT_Z = True
X_GRID = np.arange(-6, 7, 3)
Y_GRID = np.arange(-6, 7, 3)
THETA_GRID = np.arange(-12, 12, 4)


FIRST_PCD = 2
FINAL_PCD = 1200
LIDAR_PCD_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/currentDataforCVPR/template3/static/1/_out"


POSE_FILE = 'trial_pose_template3_1.json'

### Set constants and flags

In [3]:
# FIRST_PCD = 2
# FINAL_PCD = 809
# FIRST_PCD = 2
# FINAL_PCD = 879

VOXEL_SZ = 0.8
MAKE_2D = True

plt.rcParams['figure.figsize'] = [10, 10]


print("No of grid pts to evaluate per pose: {}".format(X_GRID.shape[0]*Y_GRID.shape[0]*THETA_GRID.shape[0]))

No of grid pts to evaluate per pose: 150


### Functions for pcd io

In [4]:
def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

def filter_pcd(old_pcd,
               make_2d = MAKE_2D,
               apply_downsample = True,
               apply_outlier_removal = False,
               apply_crop = True,
               invert_z = INVERT_Z,
               
               downsample_voxel_size = VOXEL_SZ,
               
               downsample_radius = 1,
               downsample_neighbors = 20,
               
               crop_min_arr = np.array([-100,-100,-2]),
               crop_max_arr = np.array([100,100,100]),
               ):
    np.random.seed(0)
    pcd = copy.deepcopy(old_pcd)
    
    if invert_z:
        some_arr = np.asarray(pcd.points)
        some_arr = np.array([(x,y,-z) for x, y, z in some_arr])
        pcd.points = o3d.utility.Vector3dVector(some_arr)
        
    if apply_crop:
        cropped_pcd = o3d.geometry.crop_point_cloud(pcd, crop_min_arr, crop_max_arr)
        pcd = cropped_pcd
    
    if apply_outlier_removal:
        denser_pcd, ind = o3d.geometry.radius_outlier_removal(pcd,
                                                              nb_points = downsample_neighbors,
                                                              radius    = downsample_radius)
        pcd = denser_pcd
    
    if make_2d:
        new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
        pcd.points = o3d.utility.Vector3dVector(new_pts)
    
        
    if apply_downsample:
        voxel_down_pcd = o3d.geometry.voxel_down_sample(pcd, voxel_size = downsample_voxel_size)
        pcd = voxel_down_pcd

    return pcd

# def make_2d(pcd):
#     new_pcd = copy.deepcopy(pcd)
#     new_pts = np.concatenate([np.asarray(pcd.points)[:,:-1],np.zeros((len(pcd.points),1))], axis=1)
#     new_pcd.points = o3d.utility.Vector3dVector(new_pts)
#     return new_pcd

def read_pcd(pcd_id):
    pcd_file = str(pcd_id) + ".ply"
    pcd = o3d.io.read_point_cloud(os.path.join(LIDAR_PCD_PATH, pcd_file))
    return pcd

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.show()
    elif where is 'mat_2d':
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.show()
        
def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

### Functions for pose estimation and scoring

In [5]:
def score_pts(src_pcd, dst_pcd, trans_arr=np.zeros(3), rot_ang=np.zeros(3)):
    NEIGHB_RADIUS = VOXEL_SZ
    
    src_tmp_pcd = copy.deepcopy(src_pcd)
    dst_tmp_pcd = copy.deepcopy(dst_pcd)
    
    transform_mat = pose2matrix(trans_arr, rot_ang)
    # Note: we apply transformation on target pcd to directly get pose (without sign change)
    dst_tmp_pcd.transform(transform_mat)
    
    score = o3d.registration.evaluate_registration(src_tmp_pcd, dst_tmp_pcd, NEIGHB_RADIUS)
    
    fit = score.fitness
    rmse = score.inlier_rmse
    n_pairs = np.asarray(score.correspondence_set).shape[0]
    return rmse, fit, n_pairs

def parallel_thread(parallel_arg):
    grid_pts, previous_pose, previous_velocity = parallel_arg[0], parallel_arg[1], parallel_arg[2]
    x_pt, y_pt, theta_pt = grid_pts[0], grid_pts[1], grid_pts[2]
    x_src, y_src, theta_src = previous_pose[0], previous_pose[1], previous_pose[2]
    x_vel, y_vel, theta_vel = previous_velocity[0], previous_velocity[1], previous_velocity[2]
    
    x = x_src + x_vel + x_pt
    y = y_src + y_vel + y_pt
    t = theta_src + theta_vel + theta_pt
    rmse, fit, n_pairs = score_pts(map_pcd, new_pcd,
                                   trans_arr = [x, y, 0],
                                   rot_ang   = [0, 0, t])
    result = {'x' : x,
              'y' : y,
              'theta' : t,
              
              'x_vel' : x_vel + x_pt,
              'y_vel' : y_vel + y_pt,
              'theta_vel' : theta_vel + theta_pt,
              
              'rmse' : rmse,
              'fit' : fit,
              'n_pairs' : n_pairs}
    return result

### Implementation

#### Create initial map

In [6]:
map_pcd = read_pcd(FIRST_PCD)
map_pcd = filter_pcd(map_pcd)
print(map_pcd)

geometry::PointCloud with 281 points.


#### Initialize pose arrays

In [7]:
pose_list = []
global_origin_pt = {'x' : 0,
                    'y' : 0,
                    'theta' : 0,
                    
                    'x_vel' : 0,
                    'y_vel' : 0,
                    'theta_vel' : 0,
                    
                    'rmse' : 0,
                    'fit' : 1,
                    'n_pairs' : np.asarray(map_pcd.points).shape[0]}
pose_list.append(global_origin_pt)

#### Iterate over all subsequent pcds for local pose estimation and building the map

In [8]:
for pcd_idx in tqdm_notebook(range(FIRST_PCD+1, FINAL_PCD+1)):
    # Read next pcd and filter it
    new_pcd = read_pcd(pcd_idx)
    new_pcd = filter_pcd(new_pcd)

    # Extract last pose
    x_src, y_src, theta_src = pose_list[-1]['x'], pose_list[-1]['y'], pose_list[-1]['theta']
    x_vel, y_vel, theta_vel = pose_list[-1]['x_vel'], pose_list[-1]['y_vel'], pose_list[-1]['theta_vel']
    
    # Extract all grid pts
    parallel_args = [([x_pt, y_pt, theta], [x_src, y_src, theta_src], [x_vel, y_vel, theta_vel])\
                     for x_pt in X_GRID for y_pt in Y_GRID for theta in THETA_GRID]
    
    # Score on all grid pts
    process_pool = Pool(cpu_count()-1)
    score_list = [each for each in process_pool.imap_unordered(parallel_thread, parallel_args)]
    process_pool.terminate()
    
    # Extract pose with best matching score
    df_score = pd.DataFrame(score_list)
    new_pose_pt = df_score.iloc[df_score.fit.argmax()].to_dict()
    pose_list.append(new_pose_pt)

    # Checkpoint the new map and pose array to files
    pd.DataFrame(pose_list).to_json(POSE_FILE, orient='records', lines=True)
    
    gc.collect()

KeyboardInterrupt: 

#### Verifying completing of SLAM iterations

In [ ]:
pcd_idx

In [ ]:
map_pcd

### Analysis

#### Look at the Map

In [ ]:
map_pcd

In [ ]:
draw_pcd(map_pcd, where='mat_3d')
draw_pcd(map_pcd, where='mat_2d')

In [ ]:
draw_pcd(map_pcd, where='opn_nb')

#### Look at the Poses

In [ ]:
df_pose = pd.DataFrame(pose_list)

In [ ]:
plt.scatter(df_pose['x'], df_pose['y'])

#### Did we hit the velocity boundary ?

In [ ]:
df_pose['x_abs'] = [0] + [df_pose.iloc[idx]['x_vel']- df_pose.iloc[idx-1]['x_vel'] for idx in range(df_pose.shape[0]) if idx != 0]
df_pose['y_abs'] = [0] + [df_pose.iloc[idx]['y_vel']- df_pose.iloc[idx-1]['y_vel'] for idx in range(df_pose.shape[0]) if idx != 0]
df_pose['theta_abs'] = [0] + [df_pose.iloc[idx]['theta_vel']- df_pose.iloc[idx-1]['theta_vel'] for idx in range(df_pose.shape[0]) if idx != 0]

In [ ]:
if df_pose['x_abs'].min() <= X_GRID[0]:
    print("Velocity boundary hit in -X direction")
    print("Expected: {} | Real : {}".format(X_GRID[0], df_pose['x_abs'].min()))

if df_pose['y_abs'].min() <= Y_GRID[0]:
    print("Velocity boundary hit in -Y direction")
    print("Expected: {} | Real : {}".format(Y_GRID[0], df_pose['y_abs'].min()))
    
if df_pose['theta_abs'].min() <= THETA_GRID[0]:
    print("Velocity boundary hit in -THETA direction")
    print("Expected: {} | Real : {}".format(THETA_GRID[0], df_pose['theta_abs'].min()))
    
if df_pose['x_abs'].max() >= X_GRID[-1]:
    print("Velocity boundary hit in +X direction")
    print("Expected: {} | Real : {}".format(X_GRID[-1], df_pose['x_abs'].max()))
    
if df_pose['y_abs'].max() >= Y_GRID[-1]:
    print("Velocity boundary hit in +Y direction")
    print("Expected: {} | Real : {}".format(Y_GRID[-1], df_pose['y_abs'].max()))

if df_pose['theta_abs'].max() >= THETA_GRID[-1]:
    print("Velocity boundary hit in +THETA direction")
    print("Expected: {} | Real : {}".format(THETA_GRID[-1], df_pose['theta_abs'].max()))
    
print("All clear?")